# Getting Started with Patra Toolkit

This notebook serves as a quickstart guide to help you learn how to:

- Load and preprocess an example image
- Perform image classification with a pretrained **ResNet50** model from PyTorch
- Generate a comprehensive Model Card using the **Patra Toolkit**

By the end of this tutorial, you'll have a Model Card (in JSON format) that captures key metadata about your model and its prediction.

---

## 1. Environment Setup

### 1.1 Install Required Packages
Run the following cell to install the required packages:
- `torch` and `torchvision` for the model and image processing
- `patra_toolkit` for creating the Model Card
- `Pillow` for image handling
- `scikit-learn` (if needed) for additional utilities

---


In [24]:
!pip install torch torchvision patra_toolkit Pillow scikit-learn

### 1.2 Import Dependencies

Below, we import the necessary libraries. We also add the repository root to `sys.path` so that the latest local version of `patra_toolkit` is imported (if needed).

---

In [25]:
import os
import sys
import io
import json
import logging
import tempfile
import requests

import torch
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from PIL import Image

# Import Patra Toolkit components
from patra_toolkit import ModelCard, AIModel

# Set logging level
logging.basicConfig(level=logging.INFO)

---
## 2. Load and Preprocess an Example Image

We'll download an example image from a URL. Then, we'll apply the same preprocessing as required by ResNet50:
- Resize to 256 pixels on the smaller side
- Center-crop to 224×224
- Convert to a tensor and normalize using the ImageNet statistics

---


In [26]:
# URL of an example image (you can choose any valid image URL)
image_url = "https://upload.wikimedia.org/wikipedia/commons/9/9a/Pug_600.jpg"  # example: a picture of a pug
response = requests.get(image_url)

# Open the image with PIL
image = Image.open(io.BytesIO(response.content)).convert("RGB")

# Define the transformations
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    # Normalization parameters from ImageNet
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

input_tensor = preprocess(image)
# Create a mini-batch as expected by the model
input_batch = input_tensor.unsqueeze(0)

---
## 3. Image Prediction with Pretrained ResNet50

We load the pretrained ResNet50 model and perform inference on the preprocessed image. We'll then decode the top prediction using the default weights.

---


In [27]:
import os
import certifi

os.environ["SSL_CERT_FILE"] = certifi.where()

# Load pretrained ResNet50 model
# torchvision.models.ResNet50_Weights.DEFAULT is available in newer torchvision releases
weights = torchvision.models.ResNet50_Weights.DEFAULT
model = torchvision.models.resnet50(weights=weights)
model.eval()  # set to evaluation mode

# Perform inference
with torch.no_grad():
    output = model(input_batch)

# Get probabilities using softmax
probabilities = F.softmax(output[0], dim=0)

# Get the top 5 predictions
top5_prob, top5_catid = torch.topk(probabilities, 5)

# Use the weights metadata to map category IDs to labels
categories = weights.meta["categories"]

## 4. Model Card Generation with Patra Toolkit

Next, we'll create a Model Card capturing key metadata about the model.
Since we're using a pretrained ResNet50, we'll record its details and top prediction.


In [28]:
# Create a ModelCard instance
mc = ModelCard(
    name="ResNet50",
    version="0.1",
    short_description="A pretrained ResNet50 image classifier",
    full_description="This model card demonstrates using a pretrained ResNet50 model from PyTorch",
    keywords="resnet50, pytorch, image classification, patra, pretrained",
    author="neelk",
    input_type="Image",
    category="classification",
    foundational_model="None",
    citation="https://doi.org/10.48550/arXiv.1512.03385"
)

# For demonstration, we set input_data as the example image URL
mc.input_data = image_url


### 4.1 Create an AIModel Instance

We attach an AIModel instance to the ModelCard with details about the model.

---

In [13]:
ai_model = AIModel(
    name="ResNet50",
    version="0.1",
    description="Pretrained ResNet50 model from torchvision for image classification.",
    owner="Neelesh Karthikeyan",
    location="",  # will be updated after model submission
    license="BSD-3 Clause",
    framework="pytorch",
    model_type="cnn",
    test_accuracy=0.75
)

# Attach the AIModel to the ModelCard
mc.ai_model = ai_model
mc.populate_requirements()

---
## 6. Submit the Model and Artifact to the Model Store

In [14]:
mc.submit_model(
    patra_server_url="http://127.0.0.1:5002",
    model=model,
    file_format="pt",
    model_store="huggingface",
    inference_label="/Users/neeleshkarthikeyan/d2i/patra-toolkit/examples/notebooks/labels.txt"
)

INFO:root:Model card validated successfully.
INFO:root:Model ID retrieved: neelk-resnet50-0.1
INFO:root:Repository credentials stored.
INFO:root:Model serialized successfully.
No files have been modified since last commit. Skipping to prevent empty commit.
INFO:root:Model uploaded at: https://huggingface.co/nkarthikeyan/neelk-resnet50-0.1/blob/main/neelk-resnet50-0.1.pt
No files have been modified since last commit. Skipping to prevent empty commit.
INFO:root:Inference label uploaded at: https://huggingface.co/nkarthikeyan/neelk-resnet50-0.1/blob/main/labels.txt
INFO:root:ModelCard submitted successfully.


{'message': 'Model card already exists', 'model_card_id': 'neelk-resnet50-0.1'}

In [18]:
mc.submit_artifact(
    artifact_path="/Users/neeleshkarthikeyan/d2i/patra-toolkit/examples/notebooks/data/camera_trap_img.JPG"
)

No files have been modified since last commit. Skipping to prevent empty commit.
INFO:root:Artifact stored at: https://huggingface.co/nkarthikeyan/neelk-resnet50-0.1/blob/main/camera_trap_img.JPG


{'artifact_location': 'https://huggingface.co/nkarthikeyan/neelk-resnet50-0.1/blob/main/camera_trap_img.JPG'}

In [23]:
mc.submit_artifact("/Users/neeleshkarthikeyan/d2i/patra-toolkit/examples/model_cards/README.json")

INFO:root:Artifact stored at: https://huggingface.co/nkarthikeyan/neelk-resnet50-0.1/blob/main/README.json


{'artifact_location': 'https://huggingface.co/nkarthikeyan/neelk-resnet50-0.1/blob/main/README.json'}